In [29]:
import os
import numpy as np
import nibabel as nib
from scipy.stats import scoreatpercentile
from scipy.optimize import curve_fit

In [30]:
patients_interictal = ["sub-patient002","sub-patient003","sub-patient005","sub-patient006","sub-patient008","sub-patient009","sub-patient012","sub-patient013","sub-patient034","sub-patient038","sub-patient041","sub-patient043","sub-patient045"]

patients_preictal = ["sub-patient002","sub-patient003","sub-patient005","sub-patient006","sub-patient008","sub-patient009","sub-patient013","sub-patient034","sub-patient038","sub-patient041","sub-patient043","sub-patient045"]

patients_ictal = ["sub-patient001","sub-patient002","sub-patient003","sub-patient004","sub-patient005","sub-patient006","sub-patient008","sub-patient009","sub-patient013","sub-patient034","sub-patient038","sub-patient041","sub-patient045","sub-patient052"]

patients_postictal = ["sub-patient001","sub-patient002","sub-patient003","sub-patient005","sub-patient006","sub-patient008","sub-patient009","sub-patient012","sub-patient034","sub-patient038","sub-patient041", "sub-patient043","sub-patient045"]


# Interictal Patients Correlations

In [31]:
path = "/home/bpereira/func/"
restpath = "ses-interictal/ses-interictal-proc.feat/maps_to_corr/"
ses = "ses-interictal"
test = ["sub-patient002"]

allsubs_happy = []
allsubs_angry = []
allsubs_vis = []
allsubs_asl = []
allsubs_cvr = []

for sub in test:

    # neuronal
    happy_path = os.path.join(path,sub,restpath,f"{sub}_{ses}_mapCOPE8_smoothed8.nii")
    angry_path = os.path.join(path,sub,restpath,f"{sub}_{ses}_mapCOPE9_smoothed8.nii")
    vis_path = os.path.join(path,sub,restpath,f"{sub}_{ses}_mapCOPE9_smoothed8.nii")

    # vascular
    asl_path = os.path.join("/home/mig_n2treatdata/derivatives/perf-preproc",sub,ses,"smoothed8_perfusion_MNI.nii")
    cvr_path = []

    # load data
    happynii = nib.load(happy_path)
    happy = happynii.get_fdata()

    angrynii = nib.load(angry_path)
    angry = angrynii.get_fdata()

    visnii = nib.load(vis_path)
    vis = visnii.get_fdata()

    aslnii = nib.load(asl_path)
    asl = aslnii.get_fdata()

    cvrnii = []
    cvr = []

    # cut fov

    cuthappy = happy[:,:,16:68]



In [32]:
print(happy.shape)
print(cuthappy.shape)
print(asl.shape)

(91, 109, 91)
(91, 109, 52)
(91, 109, 91)
